# Módulos

In [1]:
import os # Arquivos
import pandas as pd # Dados
import multiprocessing as mp # Magia negra

# Modelagem
from sklearn.feature_extraction.text import CountVectorizer # Tokenização.
from sklearn.decomposition import LatentDirichletAllocation # Latent Dirichlet Allocatoin.

# API da Gemini
import google.generativeai as genai

# Visualização do LDA
import pyLDAvis
from pyLDAvis import prepare

# Cronômetro
from time import perf_counter

# Diretório de referência
diretório = os.getcwd() 
diretório_anterior = '\\'.join(diretório.split('\\')[:-1])
diretório_dos_corpus = diretório_anterior + '\\' + '1. Coleta e tratamento\\bases para o LDA'

# Funções

In [2]:
def distribuir_tópicos(seleção, recorte, 
                       learning_method, 
                       n_components, 
                       doc_topic_prior = None, topic_word_prior = None,
                       learning_decay = .75, learning_offset = None, batch_size = None,
                       random_state = 69, n_jobs = mp.cpu_count()):
    '''Roda o modelo de LDA e retorna as distribuições de tópicos dos documentos.'''

    # Leitura da fonte de dados
    arquivo_do_corpus = f'{seleção} - {recorte}.parquet'
    df = pd.read_parquet(diretório_dos_corpus + '\\' + arquivo_do_corpus)
    df = df[df.corpus != ''].copy()
    
    # Vetorização do corpus
    vetorizador = CountVectorizer()
    corpus = vetorizador.fit_transform(df.corpus)
    
    # Latent Dirichlet Allocation
    if learning_method == 'online':
        n_docs = corpus.shape[0] # Tamanho do corpus para parâmetros ajustáveis...
        learning_offset =  int(n_docs*0.001) if learning_offset == None else learning_offset
        batch_size =  max(128, int(n_docs*0.01)) if batch_size == None else batch_size

        modelo_de_LDA = LatentDirichletAllocation(n_components = n_components, 
                                                  learning_method = learning_method, 
                                                  doc_topic_prior = doc_topic_prior, topic_word_prior = topic_word_prior,
                                                  learning_decay = learning_decay, learning_offset = learning_offset, batch_size = batch_size,
                                                  random_state = random_state, n_jobs = n_jobs)
    else: # 'batch'
        modelo_de_LDA = LatentDirichletAllocation(n_components = n_components, 
                                                  learning_method = learning_method, 
                                                  doc_topic_prior = doc_topic_prior, topic_word_prior = topic_word_prior,
                                                  random_state = random_state, n_jobs = n_jobs)
    modelo_de_LDA.fit(corpus)
    
    # Distribuição de tópicos por documento
    distribuições = modelo_de_LDA.transform(corpus)
    distribuições = pd.DataFrame(distribuições, 
                                 columns = [f'T{i+1}' for i in range(n_components)], 
                                 index = df.index) # O index possui a mesma ordenação que as distribuições..!
    
    # Output, finalmente
    resultados = pd.concat([df, distribuições], axis = 1)
    return vetorizador, corpus, modelo_de_LDA, resultados

In [3]:
def amostrar_tópicos(vetorizador, modelo_de_LDA, quantidade_de_palavras):
    '''Amostra determinada quantidade das principais palavras de cada tópico.'''
    
    nomes_das_palavras = vetorizador.get_feature_names_out()
    
    palavras_relevantes_por_tópico = {}
    for índice_do_tópico, vetor_do_tópico in enumerate(modelo_de_LDA.components_):
        índices_das_maiores_contribuições = vetor_do_tópico.argsort()[::-1][:quantidade_de_palavras]
        palavras_do_tópico = [nomes_das_palavras[índice] for índice in índices_das_maiores_contribuições]
        conjunto_de_palavras = ' '.join(palavras_do_tópico)
        palavras_relevantes_por_tópico[f'Tópico_{índice_do_tópico + 1}'] = [conjunto_de_palavras]
    
    amostras = pd.DataFrame(palavras_relevantes_por_tópico, index = ['Destaques']).T
    return amostras

In [4]:
def nomear_tópicos(amostras):
    '''Pede para a Gemini sugerir nomes preliminares aos tópicos, só pra dar para dar uma olhada a priori...'''
    
    prompt = f'''Sejam as principais palavras em ordem decrescente de importância dos seguintes tópicos:
    
    "{amostras}"
    
    Sugira um título abrangente que faça sentido semânticamente, mas sucinto, para cada um dos tópicos a seguir.
    Coloque letras maiúsculas apenas em nomes (pessoas, empresas, países etc.) e na primeira palavra.
    
    Retorne estritamente no seguinte formato:
    
    Nome do tópico;
    Nome do tópico;
    Nome do tópico;
    ...
    
    Nada mais e nada menos!
    '''
    
    genai.configure(api_key = '') # Para funcionar, você precisa criar uma conta e arranjar uma chave para a API.
    model = genai.GenerativeModel('gemini-2.0-flash')
    tópicos = model.generate_content(prompt).text.strip().replace('\n','').replace('.', '').split(';')
    while '' in tópicos:
        tópicos.remove('')
    return tópicos

# Esteira de modelagem

In [5]:
parâmetros = pd.concat([pd.read_excel('parâmetros ótimos para o método batch.xlsx'), 
                        pd.read_excel('parâmetros ótimos para o método online.xlsx')]).fillna('-')
parâmetros = parâmetros[~parâmetros.selection.isin(('umsoplaneta', 'pipelinevalor'))].copy() # Portais pequenos demais...

In [7]:
parâmetros.sort_values(['selection', 'method']) # Visualizando os parâmetros que serão utilizados.

,selection,method,n_components,learning_decay,learning_offset
4,epocanegocios,batch,40,-,-
4,epocanegocios,online,50,0.608372,494.0
0,globorural,batch,10,-,-
0,globorural,online,40,0.634183,159.0
3,revistapegn,batch,20,-,-
3,revistapegn,online,20,0.606511,348.0
1,valor,batch,50,-,-
2,valor,online,40,0.620335,904.0
2,valorinveste,batch,30,-,-
1,valorinveste,online,30,0.606511,348.0


In [69]:
recorte = 'total'
for seleção, método, n_components, learning_decay, learning_offset in zip(*parâmetros.T.values):
    for multiplicador in (1,10):

        # Parâmetros, verbose e check-points
        ajuste = 'padrão' if multiplicador == 1 else 'ajustado'
        indexação = f'{seleção} - {recorte} - {método} - {ajuste}'
        
        if f'{indexação} - proporções de tópicos.parquet' in os.listdir('resultados do LDA'):
            continue

        início = perf_counter() # Cronômetro e... Contando!
        doc_topic_prior, topic_word_prior = multiplicador*(1/n_components), (1/n_components)/multiplicador
        print(f'Modelando "{seleção}" via método {método} para {n_components} tópico(s).\nEspecificação {ajuste}: α = {doc_topic_prior:.4f}; η = {topic_word_prior:.4f}.\n')
        
        # Latent Dirichlet Allocation
        vetorizador, corpus, modelo_de_LDA, resultados = distribuir_tópicos(seleção, recorte, 
                                                                            método, 
                                                                            n_components, 
                                                                            doc_topic_prior = doc_topic_prior, # Mais tópicos por documento;
                                                                            topic_word_prior = topic_word_prior, # Menos palavras por tópico.
                                                                            learning_decay = learning_decay, learning_offset = learning_offset)

        # Pré-nomeando os tópicos automaticamente
        amostras = amostrar_tópicos(vetorizador, modelo_de_LDA, 50)
        nomes_sugeridos = nomear_tópicos(amostras['Destaques'])

        # Armazenando os resultados
            # Proporções de tópicos no tempo
        
        resultados.to_parquet(f'resultados do LDA/{indexação} - proporções de tópicos.parquet', engine = 'pyarrow')

            # Nomes dos tópicos
        tópicos = pd.DataFrame([resultados.columns[resultados.columns.str.contains('T')].tolist(), 
                                amostras['Destaques'].tolist(),
                                nomes_sugeridos], index = ['Tópico', 'Palavreado', 'Nome sugerido']).T
        tópicos['Nome oficial'] = None
        tópicos.to_excel(f'resultados do LDA/{indexação} - nomes dos tópicos.xlsx', index = False)

        # Tempo decorrido
        fim = perf_counter()
        minutos, segundos = divmod(fim-início, 60)
        print(f'Tempo decorrido: {int(minutos)}m{int(segundos):02d}s..!\n')

Modelando "globorural" via método online para 40 tópico(s).
Especificação padrão: α = 0.0250; η = 0.0250.

Tempo decorrido: 2m09s..!

Modelando "globorural" via método online para 40 tópico(s).
Especificação ajustado: α = 0.2500; η = 0.0025.

Tempo decorrido: 1m57s..!

Modelando "valorinveste" via método online para 30 tópico(s).
Especificação padrão: α = 0.0333; η = 0.0333.

Tempo decorrido: 2m08s..!

Modelando "valorinveste" via método online para 30 tópico(s).
Especificação ajustado: α = 0.3333; η = 0.0033.

Tempo decorrido: 2m11s..!

Modelando "valor" via método online para 40 tópico(s).
Especificação padrão: α = 0.0250; η = 0.0250.

Tempo decorrido: 11m37s..!

Modelando "valor" via método online para 40 tópico(s).
Especificação ajustado: α = 0.2500; η = 0.0025.

Tempo decorrido: 12m11s..!

Modelando "revistapegn" via método online para 20 tópico(s).
Especificação padrão: α = 0.0500; η = 0.0500.

Tempo decorrido: 1m43s..!

Modelando "revistapegn" via método online para 20 tópico(s)

# Modelagem artesanal

Não vai ter loop for aqui, precisa rodar seleção por seleção mesmo.

In [168]:
# Modelagem, finalmente!
seleção, recorte = 'globorural', 'total'

n_components = 30
doc_topic_prior, topic_word_prior = 1/n_components, 1/n_components
vetorizador, corpus, modelo_de_LDA, resultados = distribuir_tópicos(seleção, recorte, 
                                                                    'online', 
                                                                    n_components, 
                                                                    doc_topic_prior = 10*doc_topic_prior, # Mais tópicos por documento;
                                                                    topic_word_prior = topic_word_prior/10) # Menos palavras por tópico.

## Nomeação

In [192]:
# Nomeando os tópicos automaticamente...
amostras = amostrar_tópicos(vetorizador, modelo_de_LDA, 50)
nomes_sugeridos = nomear_tópicos(amostras['Destaques'])

# Amostragem

In [20]:
seleção, recorte, método, ajuste = 'valor', 'total', 'online', 'ajustado'
indexação = f'{seleção} - {recorte} - {método} - {ajuste}'

Se for amostragem artesanal, não rodar a célula abaixo.

In [21]:
import pandas as pd
amostragem = pd.read_parquet(f'resultados do LDA/{seleção} - {recorte} - {método} - {ajuste} - proporções de tópicos.parquet')
nomes_sugeridos = pd.read_excel(f'resultados do LDA/{indexação} - nomes dos tópicos.xlsx')['Nome oficial'].tolist()
amostragem.columns = ['url', 'data', 'corpus', 'portal'] + nomes_sugeridos

In [22]:
import random 
amostra = amostragem.iloc[[random.randint(0, len(amostragem)-1)]][['corpus'] + nomes_sugeridos].T
print(amostra.loc['corpus'].iloc[0])
(amostra.iloc[1:].sort_values(amostra.columns[0], ascending = False)*100).astype(int).astype(str) + '%'

ex-diretor petrobras entrega políticos delação premiada ex-diretor abastecimento petrobras aceitou esquema corrupção petrobras envolvendo partidos políticos empreiteiras troca judicial redução expressiva pena condenado costa pena corrupção envolvendo petrolífera réu justiça federal prisão preventiva


,519917
Operação Lava Jato,49%
Petrobras,15%
Executivos,5%
Combustíveis fósseis,3%
Eleições,3%
Mercado financeiro nacional,2%
Agenda econômica,0%
Congresso Nacional,0%
Regulação econômica e defesa da concorrência,0%
Agricultura argentina,0%
